## Importing the Data

In [48]:
# Constants
DATASET_DIR = './data/' # Datasets to be places here
SAVE_DIR = './' # Main Dir

import os
import pandas as pd
import numpy as np

# Removing empty columns and finding minimum and maximum scores given to each of the 8 sets

X = pd.read_csv(os.path.join(DATASET_DIR, 'training_set_rel3.tsv'), sep='\t', encoding='ISO-8859-1')
# Reading from tsv file (tab - separated) with Latin alphabet encoding including special symbols
y = X['domain1_score']
X = X.dropna(axis=1)
X = X.drop(columns=['rater1_domain1', 'rater2_domain1'])
Z = pd.read_excel(r'./data/essay_set_descriptions.xlsx')
minimum_scores = Z['min_domain1_score'].to_list()
minimum_scores.insert(0,-1)
maximum_scores = Z['max_domain1_score'].to_list()
maximum_scores.insert(0,-1)

In [49]:
X.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


Minimum and Maximum Scores for each essay set.

In [50]:
print(minimum_scores)
print(maximum_scores)

[-1, 2, 1, 0, 0, 0, 0, 0, 0]
[-1, 12, 6, 3, 3, 4, 4, 30, 60]


## Pre-processing of the Data

These are all helper functions used to clean and tokenize the essays.

In [51]:
# !pip install gensim
# !pip install nltk
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('popular')
import re # for regular expressions operations 
from nltk.corpus import stopwords

def essay_to_wordlist(essay_v, remove_stopwords):
    # remove the tagged labels and word tokenize the sentence
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v) # removing anything that is not alphabetic
    words = essay_v.lower().split() # turn sentence into lowercase and split it into words 
    if remove_stopwords:
        stops = set(stopwords.words("english")) # english stopwords library 
        words = [w for w in words if not w in stops] # words present in the sentence and not present in stopwords
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    # sentence tokenize the essay and call essay_to_wordlist() for word tokenization
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle') # nltk library
    raw_sentences = tokenizer.tokenize(essay_v.strip()) # call tokenizer on essay striped of spaces 
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
            # remove_stopwords carried forward to essay_to_wordlist it is a bool variable
    return sentences # will be returning list of tokenized sentences without stopwords

## Word2Vec Model

We will preprocess all essays and convert them to feature vectors and will visualize them using tensorflow

In [56]:
from gensim.models import Word2Vec # for word2vec
from gensim.test.utils import get_tmpfile

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    #  make null vectors of length = num_features for every word index
    num_words = 0.
    index2word_set = set(model.wv.index2word) # vocabulary set of model
    for word in words:
        if word in index2word_set: # for word in a sentence being in index2word_set
            num_words += 1
            featureVec = np.add(featureVec,model[word])
    if num_words:
        featureVec = np.divide(featureVec,num_words) 
        # divides every element of featureVec by num_words (to compute average) 
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    # it then calls makeFeatureVec and for every essay sentance
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    # make null vectors of length = length of essay having null vector of num_features at every index
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs
    # then returns the trained model feature vectors
    
allsentences = [] # list of all sentences
    
for essay in X['essay']:
# obtaining all sentences from the essays.
    allsentences += essay_to_sentences(essay, remove_stopwords = True)
            
# initializing variables for word2vec model.
num_features = 300 # vector length
min_word_count = 40 # to be considered for vectorisation
num_workers = 8 # working cores
context = 10
downsampling = 1e-3 # compressing

# for visualization of vectors
visualmodel = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

path = get_tmpfile("word2vecvisual.model") # path for the model 

visualmodel.init_sims(replace=True) # normalized model
# different ways to save model
visualmodel.wv.save_word2vec_format('word2vecvisual.txt', binary=False)
visualmodel.wv.save_word2vec_format('word2vecvisual.bin', binary=True)
visualmodel.save("word2vecvisual,model")

2020-09-25 02:32:26,625 - dictionary - INFO - adding document #0 to Dictionary(0 unique tokens: [])
2020-09-25 02:32:26,630 - dictionary - INFO - built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2020-09-25 02:32:26,636 - word2vec - INFO - collecting all words and their counts
2020-09-25 02:32:26,644 - word2vec - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-09-25 02:32:26,734 - word2vec - INFO - PROGRESS: at sentence #10000, processed 81383 words, keeping 7084 word types
2020-09-25 02:32:26,802 - word2vec - INFO - PROGRESS: at sentence #20000, processed 162931 words, keeping 10507 word types
2020-09-25 02:32:26,841 - word2vec - INFO - PROGRESS: at sentence #30000, processed 243216 words, keeping 13069 word types
2020-09-25 02:32:26,879 - word2vec - INFO - PROGRESS: at sentence #40000, processed 327952 words, keeping 15868 word types
2020-09-25 02:32:26,956 - word2

In [54]:
import io # for input output

# for visualization of a stored model
'''from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format(SAVE_DIR+'/word2vecmodel.bin', binary=True)'''

# files needed for tensorboard
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

# write meta file and vector file
for index in range(len(model.index2word)): # for every word in vocab
    word = model.index2word[index]
    vec = model.vectors[index]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

'''Open http://projector.tensorflow.org/
   Click “Load Data” button from the left menu.
   Select “Choose file” in “Load a TSV file of vectors.” and choose “vecs.tsv” file.
   Select “Choose file” in “Load a TSV file of metadata.” and choose “meta.tsv” file.
   The model has been visualized in 3D/2D.
   It is done by choosing variable with higher variance as a dimension from the vector'''



2020-09-25 02:22:31,709 - utils_any2vec - INFO - loading projection weights from .//word2vecmodel.bin
2020-09-25 02:22:31,788 - utils_any2vec - INFO - loaded (2591, 300) matrix from .//word2vecmodel.bin


## Defining the model 

Here we define a 2-Layer LSTM Model. 

Note that instead of using sigmoid activation in the output layer we will use
Relu since we are not normalising training labels.

In [6]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

## Training Phase

Now we train the model on the dataset.

We will use 5-Fold Cross Validation and measure the Quadratic Weighted Kappa for each fold.
We will then calculate Average Kappa for all the folds.

In [7]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score


cv = KFold(n_splits=5, shuffle=True)
cv.get_n_splits(X)
results = []
y_pred_list = []

count = 1
for traincv, testcv in cv.split(X):
    print("\n--------Fold {}--------\n".format(count))
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    sentences = []
    
    for essay in train_essays:
            # Obtaining all sentences from the training essays.
            sentences += essay_to_sentences(essay, remove_stopwords = True)
            
    # Initializing variables for word2vec model.
    num_features = 300 
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3

    print("Training Word2Vec Model...")
    model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.txt', binary=False)

    clean_train_essays = []
    
    # Generate training and testing data word vectors.
    for essay_v in train_essays:
        clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
    clean_test_essays = []
    for essay_v in test_essays:
        clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
    testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
    y_pred = lstm_model.predict(testDataVecs)
    
    # Save any one of the 8 models.
    if count == 5:
         lstm_model.save('./model_weights/final_lstm.h5')
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print("Kappa Score: {}".format(result))
    results.append(result)

    count += 1
        

4108 - mae: 1.7962
Epoch 26/50
163/163 [==============================] - 4s 23ms/step - loss: 11.1056 - mae: 1.8006
Epoch 27/50
163/163 [==============================] - 3s 18ms/step - loss: 10.4238 - mae: 1.7695
Epoch 28/50
163/163 [==============================] - 4s 26ms/step - loss: 10.0927 - mae: 1.7582
Epoch 29/50
163/163 [==============================] - 3s 20ms/step - loss: 10.7142 - mae: 1.7908
Epoch 30/50
163/163 [==============================] - 3s 19ms/step - loss: 10.1332 - mae: 1.7479
Epoch 31/50
163/163 [==============================] - 3s 20ms/step - loss: 10.1616 - mae: 1.7618
Epoch 32/50
163/163 [==============================] - 3s 21ms/step - loss: 9.7996 - mae: 1.7333
Epoch 33/50
163/163 [==============================] - 3s 21ms/step - loss: 10.0107 - mae: 1.7396
Epoch 34/50
163/163 [==============================] - 4s 23ms/step - loss: 9.4298 - mae: 1.7278
Epoch 35/50
163/163 [==============================] - 3s 21ms/step - loss: 9.5925 - mae: 1.7309
Epoc

The Avg. Kappa Score is 0.961 which is the highest we have ever seen on this dataset.

In [8]:
print("Average Kappa score after a 5-fold cross validation: ",np.around(np.array(results).mean(),decimals=4))

Average Kappa score after a 5-fold cross validation:  0.9588
